In [1]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os.path
from shutil import copyfile
from datetime import datetime

# Directory containing the files
dir_raw = "C:/Users/saidul.haq/OneDrive - World Food Programme/Saidul/1.mVAM dashboard project/BD mVAM data/2024 raw"
dir_analyzed = "C:/Users/saidul.haq/OneDrive - World Food Programme/Saidul/1.mVAM dashboard project/BD mVAM data/analyzed"
dir_main = "C:/Users/saidul.haq/OneDrive - World Food Programme/Saidul/1.mVAM dashboard project/BD mVAM data"
# Improt Variable Mapping Workbook 
VarMapping = "C:/Users/saidul.haq/OneDrive - World Food Programme/Saidul/1.mVAM dashboard project/automation/variableMappingWorkbook.xlsx"
# Read Excel data into a DataFrame
VarMapping = pd.read_excel(VarMapping, sheet_name='Sheet1')
# Print the data
print(VarMapping)

# Path to the Excel file
excel_file = "C:/Users/saidul.haq/OneDrive - World Food Programme/Saidul/1.mVAM dashboard project/BD mVAM data/2024 raw/Round26Sep2024.xlsx"

# Read Excel data into a DataFrame
data = pd.read_excel(excel_file, sheet_name='data')
# See the variable types and shape of data
print(data['CallDispo'].dtypes)
print(data['consent'].dtypes)
# Print the shape of the data
print("Shape of the data:", data.shape)

# Convert 'consent' column from float64 to integer64
data['consent']= data['consent'].astype('Int64')

# filter the data where we drop all not responded rows
data1 = data[(data['CallDispo']==1) & (data['consent']==1)]

# Print the shape of the data1
print("Shape of the data:", data1.shape)

# Keep only unique IDs in the 'UiD_phonebook' column
data1 = data1.drop_duplicates(subset=['UiD_phonebook'])

# Print the shape of the data1
print("Shape of the data:", data1.shape)

# Now drop the unnessary VArible from the data
# Step 2: Extract variable names from the Excel file
variables_drop_row = VarMapping['DataToDropFromRaw'].dropna().tolist() 
data1 = data1.drop(columns=variables_drop_row, errors='ignore')

# drop comments column
columns = data1.columns.tolist()
drop_com = [col for col in columns if 'comments.' in col]
data1.drop(columns=drop_com)

rename_mapping = dict(zip(VarMapping['ExistingName'], VarMapping['NewName']))
# Rename varible in the data
data1.rename(columns=rename_mapping, inplace=True)

print(data1.columns.tolist())
data1['HH_mainincomesource'].value_counts()

name = {
    10: "Barishal",
    20: "Chattogram",
    30: "Dhaka",
    40: "Khulna",
    45: "Mymensingh",
    50: "Rajshahi",
    55: "Rangpur",
    60: "Sylhet"
}
# Mapping the labels
data1['DIVISION_Name'] = data1['DIVISION_Name'].map(name)

data1['DIVISION_Name'].value_counts()

# creat Incomegroup variables
data1['Incomegroup'] = data1['HH_mainincomesource'].replace({
    1:1, 2:1, 5:1,
    3:2, 4:2, 12:2, 13:2, 17:2,
    6:3, 7:3, 8:3, 9:3, 10:3, 11:3, 14:3, 15:3, 16:3, -88:3
}).map({1: "High Income Group", 2: "Medium Income Group", 3: "Low Income Group"})
data1['Incomegroup'].value_counts()

# Household Income Change Type
income = {1: 'Income decreased', 2:'Income increased', 3:'No Change'}
data1['HH_incomechangetype']= data1['HH_incomechangetype'].replace(income)
data1['HH_incomechangetype'].value_counts()

# Household Expenditure Change Type
expenditure = {1: 'Expenditure decreased', 2: 'Expenditure increased', 3:'No change'}
data1['HH_expchangetype'] = data1['HH_expchangetype'].replace(expenditure)
data1['HH_expchangetype'].value_counts()

# compute FCSCat VAriable
columns_to_recode = ['FCSStap', 'FCSVeg', 'FCSFruit', 'FCSPr', 'FCSPulseNut', 'FCSDairy', 'FCSFat', 'FCSSugar']
# Recode the specified columns
data[columns_to_recode] = data[columns_to_recode].replace({'.': 0})

# Compute FCS
data1['FCS'] = (data1['FCSStap']*2)+ (data1['FCSVeg']*1)+ (data['FCSFruit'] * 1) + (data['FCSPr'] * 4) + (data['FCSPulseNut'] * 3) + (data['FCSDairy'] * 4) + (data['FCSFat'] * 0.5) + (data['FCSSugar'] * 0.5)

data1['FCSCat_aux']=0

data1.loc[data1['FCS']<=28, 'FCSCat_aux']=1
data1.loc[(data1['FCS']>28) & (data1['FCS']<=42), 'FCSCat_aux']=2
data1.loc[data1['FCS']>42, 'FCSCat_aux']=3

fcs = {1: 'Poor', 2: 'Borderline', 3: 'Acceptable'}

data1['FCSCat'] = data1['FCSCat_aux'].replace(fcs)
data1['FCSCat'].value_counts()


# Compute rCSI
data1['rCSI']=  (data1['rCSILessQlty']*1) + (data1['rCSIBorrow']*2) + (data1['rCSIMealNb']*1) + (data1['rCSIMealSize']*1) + (data1['rCSIMealAdult']*3)

data1['rCSICat_aux']=0
data1.loc[data1['rCSI']<= 3, 'rCSICat_aux']=1
data1.loc[(data1['rCSI']>3) & (data1['rCSI']<=18), 'rCSICat_aux']=2
data1.loc[data1['rCSI']>18, 'rCSICat_aux']=3

rcs = {1: "No or low coping",  2: "Stress coping" ,3: "Crisis and above crisis coping"}

data1['rCSICat'] = data1['rCSICat_aux'].replace(rcs)


# Compute FCS_CARI
data1['FCS_CARI_aux']=0
data1.loc[data1['FCSCat']=='Acceptable', 'FCS_CARI_aux']=1
data1.loc[(data1['FCSCat']=='Acceptable') & (data1['rCSICat']=='Crisis and above crisis coping'), 'FCS_CARI_aux']=2
data1.loc[data1['FCSCat']=='Borderline', 'FCS_CARI_aux']=3
data1.loc[data1['FCSCat']=='Poor', 'FCS_CARI_aux']=4

fcs_cari = {1: 'Acceptable', 2: 'Acceptable food consumption but severe food coping', 3: 'Borderline', 4: 'Poor'}

data1['FCS_CARI'] = data1['FCS_CARI_aux'].replace(fcs_cari)
data1['FCS_CARI'].value_counts()




## *1. STRESS STRATEGIES: sold household assets, purchased on credit, spent savings, borrowed money

data1['stress_coping_EN'] = ((data1['LcsEN_stress_DomAsset'] == 2) | 
                            (data1['LcsEN_stress_DomAsset'] == 3) |
                            (data1['LcsEN_stress_CrdtFood'] == 2) |
                            (data1['LcsEN_stress_CrdtFood'] == 3) |
                            (data1['LcsEN_stress_Saving'] == 2) |
                            (data1['LcsEN_stress_Saving'] == 3) |
                            (data1['LcsEN_stress_BorrowCash'] == 2) |
                            (data1['LcsEN_stress_BorrowCash'] == 3)).astype(int)

data1['stress_coping_EN'].value_counts()

# *2. CRISIS STRATEGIES: sold productive assets, reduced expenses for health/education, withdrew 
# children from school 

data1['crisis_coping_EN'] = ((data1['LcsEN_crisis_ProdAsset'] == 2) | 
                            (data1['LcsEN_crisis_ProdAsset'] == 3) |
                            (data1['LcsEN_crisis_HealthEdu'] == 2) |
                            (data1['LcsEN_crisis_HealthEdu'] == 3) |
                            (data1['LcsEN_crisis_AgriInput'] == 2) |
                            (data1['LcsEN_crisis_AgriInput'] == 3) |
                            (data1['LcsEN_crisis_Seed'] == 2) |
                            (data1['LcsEN_crisis_Seed'] == 3)).astype(int)
data1['crisis_coping_EN'].value_counts()

# *3. EMERGENCY STRATEGIES: sold house/land, begged, engaged in illegal income activities

data1['emergency_coping_EN'] = ((data1['LcsEN_em_Migrate'] == 2) | 
                                (data1['LcsEN_em_Migrate'] == 3) |
                                (data1['LcsEN_em_Begged'] == 2) |
                                (data1['LcsEN_em_Begged'] == 3) |
                                (data1['LcsEN_em_IllegalAct'] == 2) |
                                (data1['LcsEN_em_IllegalAct'] == 3) |
                                (data1['LcsEN_em_ResAsset'] == 2) |
                                (data1['LcsEN_em_ResAsset'] == 3)).astype(int)

data1['emergency_coping_EN'].value_counts()
# compute Max coping 
data1['Max_coping_behaviorEN_aux'] = 1
data1.loc[data1['stress_coping_EN'] == 1, 'Max_coping_behaviorEN_aux'] = 2
data1.loc[data1['crisis_coping_EN'] == 1, 'Max_coping_behaviorEN_aux'] = 3
data1.loc[data1['emergency_coping_EN'] == 1, 'Max_coping_behaviorEN_aux'] = 4

# Assuming you have a dictionary to map the labels
label_max_cop = {
    1: "HH not adopting coping strategies",
    2: "Stress coping strategies",
    3: "Crisis coping strategies",
    4: "Emergency coping strategies"
}

# Mapping the labels
data1['Max_coping_behaviorEN'] = data1['Max_coping_behaviorEN_aux'].map(label_max_cop)

data1['Max_coping_behaviorEN'].value_counts()

# Household Income Change 

# First, initialize 'HH_incomechange' as 0
data1['HH_incomechange_aux'] = 0

# Now, apply the conditions using pandas' loc method
data1.loc[data1['HH_incomechangetype'] == 'Income increased', 'HH_incomechange_aux'] = 1
data1.loc[data1['HH_incomechangetype'] == 'No Change', 'HH_incomechange_aux'] = 2
data1.loc[data1['HHIncChTypeDec'] == 3, 'HH_incomechange_aux'] = 3
data1.loc[data1['HHIncChTypeDec'] == 2, 'HH_incomechange_aux'] = 4
data1.loc[data1['HHIncChTypeDec'] == 1, 'HH_incomechange_aux'] = 5

# Display the frequency table of 'HH_incomechange'
print(data1['HH_incomechange_aux'].value_counts())

# Assuming you have a dictionary to map the labels
label_income = {
    1: "Income Increased",
    2: "No Change",
    3: "Reduced by less than 25%",
    4: "Reduced by more than 25% and less than 50%",
    5: "Reduced by more than 50%"
}

# Mapping the labels
data1['HH_incomechange'] = data1['HH_incomechange_aux'].map(label_income)

data1['HH_incomechange'].value_counts()

# creat Incomegroup variables
data1['HHIncFirst'] = data1['HH_mainincomesource'].replace({
    1:1, 2:1, 3:1, 4:1, 5:1, 17:1,
    6:2, 7:2, 8:2, 9:2, 10:2,
    11:3, 12:3, 13:3, 14:3, 15:3, 16:3, -88:3})


# First, initialize 'CARI_Inc' as 0
data1['CARI_Inc_aux'] = 0

# Now, apply the conditions using pandas' loc method
data1.loc[(data1['HHIncFirst'] == 1) & (data1['HH_incomechange_aux'] < 3), 'CARI_Inc_aux'] = 1
data1.loc[(data1['HHIncFirst'] == 1) & (data1['HH_incomechange_aux'] >= 3), 'CARI_Inc_aux'] = 2
data1.loc[(data1['HHIncFirst'] == 2) & (data1['HH_incomechange_aux'] < 3), 'CARI_Inc_aux'] = 2
data1.loc[(data1['HHIncFirst'] == 2) & (data1['HH_incomechange_aux'] >= 3), 'CARI_Inc_aux'] = 3
data1.loc[data1['HHIncFirst'] == 3, 'CARI_Inc_aux'] = 4

# Display the frequency table of 'CARI_Inc'
print(data1['CARI_Inc_aux'].value_counts())

# Assuming you have a dictionary to map the labels
label_cari_in = {
    1: "Regular employment (formal labour or self-employed) – income no change or increased",
    2: "Regular employment but reduced income or informal source, income no change/ no decrease",
    3: "Informal labour /remittances but reduced income",
    4: "No income, dependent on assistance or support"
}

# Mapping the labels
data1['CARI_Inc'] = data1['CARI_Inc_aux'].map(label_cari_in)

data1['CARI_Inc'].value_counts()

## calculating rCARI

data1['rCARI_mean'] = (data1['FCSCat_aux'] * 0.25) + (data1['rCSICat_aux'] * 0.25) + (data1['CARI_Inc_aux'] * 0.25) + (data1['Max_coping_behaviorEN_aux'] * 0.25)


data1['rCARI_aux'] = 0

# Now, apply the conditions using pandas' loc method
data1.loc[data1['rCARI_mean'] < 1.5, 'rCARI_aux'] = 1
data1.loc[(data1['rCARI_mean'] >= 1.5) & (data1['rCARI_mean'] <= 2.25), 'rCARI_aux'] = 2
data1.loc[(data1['rCARI_mean'] >= 2.5) & (data1['rCARI_mean'] <= 3.25), 'rCARI_aux'] = 3
data1.loc[data1['rCARI_mean'] >= 3.5, 'rCARI_mean'] = 4


# Assuming you have a dictionary to map the labels
label_rCARI = {
    1: "Food secure",
    2: "Marginally food secure",
    3: "Moderately food insecure",
    4: "Severely food insecure"
}

# Mapping the labels
data1['rCARI'] = data1['rCARI_aux'].map(label_rCARI)

data1['rCARI'].value_counts()

# compute aggregates of key micronutrient consumption – vitamin, iron and protein 
col_recode = ['FCSStap', 'FCSVeg', 'FCSFruit', 'FCSPr', 'FCSPulseNut', 'FCSDairy', 'FCSFat', 'FCSSugar']
data1[col_recode] = data1[col_recode].replace('.',0)
col_recode2 = ['FCSDairy', 'FCSPr', 'FCSPrMeatF', 'FCSPrMeatO', 'FCSPrFish', 'FCSDriedFish', 'FCSPrEgg', 'FCSVeg', 'FCSVegOrg', 'FCSVegGre', 'FCSFruit', 'FCSFruitOrg', 'FCSFruitsOth']
data1[col_recode2] = data1[col_recode2].replace('.',0)

data1['FGVitA'] = data1['FCSDairy'] + data1['FCSPrMeatO'] + data1['FCSPrEgg'] + data1['FCSVegOrg'] + data1['FCSVegGre'] + data1['FCSFruitOrg'] + data1['FCSFruitsOth']
data1['FGProtein'] = data1['FCSPulseNut'] + data1['FCSDairy'] + data1['FCSPrMeatF'] + data1['FCSPrMeatO'] + data1['FCSPrFish'] + data1['FCSPrEgg']
data1['FGHIron'] = data1['FCSPrMeatF'] + data1['FCSPrMeatO'] + data1['FCSPrFish']

# Recoding the key micronutrient consumptions
bins = [0,1,6,data1['FGVitA'].max()]
bins1 = [0,1,6,data1['FGProtein'].max()]
bins2 = [0,1,6,data1['FGHIron'].max()]
labels = [1,2,3]

data1['FGVitACat_aux'] = pd.cut(data1['FGVitA'], bins=bins, labels=labels, include_lowest=True)
data1['FGProteinCat_aux'] = pd.cut(data1['FGProtein'], bins = bins1, labels=labels, include_lowest=True)
data1['FGHIron_aux'] = pd.cut(data1['FGHIron'], bins = bins2, labels=labels, include_lowest=True)

# Add value labels 
# Assuming you have a dictionary to map the labels
label_nut = {
    1: "0 days",
    2: "1-6 days",
    3: "7 days"
}

# Mapping the labels
data1['FGVitACat'] = data1['FGVitACat_aux'].map(label_nut)
data1['FGProteinCat'] = data1['FGProteinCat_aux'].map(label_nut)
data1['FGHIronCat'] = data1['FGHIron_aux'].map(label_nut)

## show the values
col_print = ['FGVitA','FGProtein', 'FGHIron','FGVitACat', 'FGProteinCat', 'FGHIronCat']
# iterate over each column
for column in col_print:
    print(data1[column].value_counts())
    
# Create two variable dummy
data1['HH_expenditurechange_3months'] = None
data1['HH_inomechange_6months'] = None

      sl DataToDropFromRaw       KeepInAnalyzed     ExistingName  \
0      1         starttime  DataCollection_Date             Date   
1      2           endtime        DIVISION_Name  ADMIN1_Division   
2      3             today        DISTRICT_Name  ADMIN2_District   
3      4          deviceid         UPAZILA_Name   ADMIN3_Upazila   
4      5             simid  Area_RuralUrbanCity     HHurbanRural   
..   ...               ...                  ...              ...   
130  131               NaN                  NaN     HHConcern/12   
131  132               NaN                  NaN     HHConcern/13   
132  133               NaN                  NaN     HHConcern/14   
133  134               NaN                  NaN     HHConcern/15   
134  135               NaN                  NaN    HHConcern/-88   

                           NewName  
0              DataCollection_Date  
1                    DIVISION_Name  
2                    DISTRICT_Name  
3                     UPAZILA_Name 

# Create the varibales for KPI

In [2]:
print("Shape of the data:", data1.shape)

Shape of the data: (1181, 215)


# working with reshaping data

In [3]:
file_save = "C:/Users/saidul.haq/OneDrive - World Food Programme/Saidul/1.mVAM dashboard project/BD mVAM data/pivot data/Pivot.xlsx"

Pivot_data =data1[['DataCollection_Date','DIVISION_Name','DISTRICT_Name', 'DISTRICT_Name', 'Incomegroup', 'HHShock_6MNo0', 'HHShock_6MLossofemploymentre', 'HHShock_6MSicknesshealthexp', 'HHShock_6MDeathofhouseholdme', 'HHShock_6MHighfoodprices4', 'HHShock_6MHighfuelpricetrans', 'HHShock_6MNaturalhazardsfloo', 'HHShock_6MPoorharvest7', 'HHShock_6MElectricitygascuts', 'HHShock_6MInsecuritythefts9', 'HHShock_6MIrregularunsafedrin', 'HHShock_6MLackofaccesstocre', 'HHShock_6MDebttoreinburse12', 'HHShock_6MRentpayment13', 'HHShock_6MEnvironmentalproblem', 'HHShock_6MOthers88']]

## Convert the data type of new table
print(Pivot_data.dtypes)
Pivot_data = Pivot_data.astype(str)
print(Pivot_data.dtypes)

## combine somes columns to keep it Id variable in the reshpaed table
Pivot_data['Date_Div_Income'] = Pivot_data[['DataCollection_Date', 'DIVISION_Name', 'DISTRICT_Name', 'Incomegroup']].agg(','.join, axis=1)
Pivot_data

## Now reshpae the table
reshaped_HHShock = pd.melt(Pivot_data, id_vars= 'Date_Div_Income',  value_vars=['HHShock_6MNo0', 'HHShock_6MLossofemploymentre', 'HHShock_6MSicknesshealthexp', 'HHShock_6MDeathofhouseholdme', 'HHShock_6MHighfoodprices4', 'HHShock_6MHighfuelpricetrans', 'HHShock_6MNaturalhazardsfloo', 'HHShock_6MPoorharvest7', 'HHShock_6MElectricitygascuts', 'HHShock_6MInsecuritythefts9', 'HHShock_6MIrregularunsafedrin', 'HHShock_6MLackofaccesstocre', 'HHShock_6MDebttoreinburse12', 'HHShock_6MRentpayment13', 'HHShock_6MEnvironmentalproblem', 'HHShock_6MOthers88'])

print(reshaped_HHShock.dtypes)

reshaped_HHShock[['Date','Division', 'District', 'District_Name', 'IncomeGroup']] = reshaped_HHShock.Date_Div_Income.str.split(",", expand=True)

reshaped_HHShock[['aux','HH_Shock']] = reshaped_HHShock.variable.str.split("_", expand=True)
HH_Shock=reshaped_HHShock[['Date','Division', 'IncomeGroup', 'HH_Shock', 'value']]

HH_Shock['IncomeGroup_count']= HH_Shock['IncomeGroup'].groupby(HH_Shock['IncomeGroup']).transform('count')

HH_Shock.to_excel(file_save, sheet_name="Mar24", index=False)

DataCollection_Date               datetime64[ns]
DIVISION_Name                             object
DISTRICT_Name                            float64
DISTRICT_Name                            float64
Incomegroup                               object
HHShock_6MNo0                            float64
HHShock_6MLossofemploymentre             float64
HHShock_6MSicknesshealthexp              float64
HHShock_6MDeathofhouseholdme             float64
HHShock_6MHighfoodprices4                float64
HHShock_6MHighfuelpricetrans             float64
HHShock_6MNaturalhazardsfloo             float64
HHShock_6MPoorharvest7                   float64
HHShock_6MElectricitygascuts             float64
HHShock_6MInsecuritythefts9              float64
HHShock_6MIrregularunsafedrin            float64
HHShock_6MLackofaccesstocre              float64
HHShock_6MDebttoreinburse12              float64
HHShock_6MRentpayment13                  float64
HHShock_6MEnvironmentalproblem           float64
HHShock_6MOthers88  

C:\Users\saidul.haq\AppData\Local\Temp\ipykernel_33740\4566025.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HH_Shock['IncomeGroup_count']= HH_Shock['IncomeGroup'].groupby(HH_Shock['IncomeGroup']).transform('count')


# Working with main data sources

In [4]:
excel_file_path ="C:/Users/saidul.haq/OneDrive - World Food Programme/Saidul/1.mVAM dashboard project/BD mVAM data/BGD_WFP_mVAM_round_combine_tableau_datasource_new.xlsx"
# Read Excel data into a DataFrame
source_data= pd.read_excel(excel_file_path, sheet_name='BGD_WFP_mVAM_round_combine', engine='openpyxl')


In [5]:
# Now keep the variable for final main data sources for tableau 
variables_keep = VarMapping['KeepInAnalyzed'].dropna().tolist() 

# Filter out the if any column that do not exist in the data1, it's beacuse the data collection form changes time to time
existing_cols = [col for col in variables_keep if col in data1.columns]

# Create the final dataFrame with only the existing columns
data_main = data1[existing_cols].copy()


In [6]:
# Now keep the variable for final main data sources for tableau 
variables_to_keep = VarMapping['KeepInAnalyzed'].dropna().tolist()
data_source = source_data[variables_to_keep].copy()
print("Shape of the data:", data_source.shape)

Shape of the data: (31064, 47)


In [7]:
# Append the new data to the existing data
combined_df = pd.concat([data_source, data_main], axis=0, ignore_index=True)
# Print the shape of the data
print("Shape of the data:", combined_df.shape)

Shape of the data: (32245, 47)


In [8]:
 #Write the combined data to the same Excel file, replacing the existing data
combined_df.to_excel(excel_file_path, sheet_name="BGD_WFP_mVAM_round_combine", index=False)

In [9]:
## show the values
col_print = ['rCARI','rCSI','rCARI_mean','rCARI_aux', 'FCSCat_aux','rCSICat_aux', 'CARI_Inc_aux', 'Max_coping_behaviorEN_aux']
# iterate over each column
for column in col_print:
    print(data1[column].value_counts())

rCARI
Marginally food secure      916
Moderately food insecure    241
Food secure                  23
Name: count, dtype: int64
rCSI
0.0     452
2.0     157
3.0     136
1.0      98
4.0      76
5.0      58
6.0      52
7.0      49
9.0      16
12.0     15
10.0     12
8.0       9
13.0      7
14.0      6
16.0      6
18.0      6
11.0      5
15.0      4
22.0      3
20.0      3
17.0      2
24.0      2
21.0      2
19.0      2
27.0      1
23.0      1
29.0      1
Name: count, dtype: int64
rCARI_mean
2.25    256
2.00    237
1.50    212
1.75    211
2.50    163
2.75     70
1.25     23
3.00      8
4.00      1
Name: count, dtype: int64
rCARI_aux
2    916
3    241
1     23
0      1
Name: count, dtype: int64
FCSCat_aux
3    831
2    344
1      6
Name: count, dtype: int64
rCSICat_aux
1    843
2    323
3     15
Name: count, dtype: int64
CARI_Inc_aux
1    444
2    388
3    264
4     85
Name: count, dtype: int64
Max_coping_behaviorEN_aux
3    449
2    365
1    348
4     19
Name: count, dtype: int64


In [10]:
data1['rCARI'].value_counts()


rCARI
Marginally food secure      916
Moderately food insecure    241
Food secure                  23
Name: count, dtype: int64